In [1]:
import os
import pickle
import numpy as np
from scipy.io import loadmat

import scfw.cov_estimation as ce
from scfw.frank_wolfe import frank_wolfe
from scfw.scopt import scopt

In [35]:
data_folder = '../data/'
problem_name = 'Lymph.mat'
results = {problem_name: {}}
results_folder = './results/'
results_file = os.path.join(results_folder, 'cov_estimation', problem_name + '.pckl')
if os.path.isfile(results_file):
    with open(results_file, "rb") as f:
        try:
            results = pickle.load(f)
        except Exception:
            pass 

In [43]:
# load data
data = loadmat(os.path.join(data_folder, problem_name))
A = data['S']
p = A.shape[0]
X_0 = np.eye(p) / p


data    


{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNX86, Created on: Wed Nov  4 11:31:29 2009',
 '__version__': '1.0',
 '__globals__': [],
 'S': array([[ 1.        ,  0.19824624,  0.22726562, ...,  0.10121813,
          0.1814433 , -0.00468225],
        [ 0.19824624,  1.        ,  0.84999424, ..., -0.18705976,
          0.18268728, -0.05512259],
        [ 0.22726562,  0.84999424,  1.        , ..., -0.02605269,
          0.17741383, -0.17062154],
        ...,
        [ 0.10121813, -0.18705976, -0.02605269, ...,  1.        ,
         -0.20525492, -0.08542514],
        [ 0.1814433 ,  0.18268728,  0.17741383, ..., -0.20525492,
          1.        , -0.05129423],
        [-0.00468225, -0.05512259, -0.17062154, ..., -0.08542514,
         -0.05129423,  1.        ]]),
 'data': array([[ 0.16513519, -0.00536025, -0.09354758, ...,  2.67360565,
         -0.64652602, -0.76635604],
        [-0.72074998, -0.9146291 , -0.77636407, ...,  0.81429174,
         -1.2176625 , -0.93057636],
        [-0.564417

## Parameters initialization

In [37]:
#first set of parameters
Mf = 2
nu = 3
#running parameters

terminate_tol = 1e-10

FW_params={
    'iter_FW': 100,
    'line_search_tol': 1e-10,
}


sc_params={
    #parameters for SCOPT
    'iter_SC': 1000,
    'Lest': 'estimate', #estimate L
    'use_two_phase': True,
    #FISTA parameters
    'fista_type': 'mfista',
    'fista_tol': 1e-5,
    'fista_iter': 1000,
    #Conjugate Gradient Parameters
    'conj_grad_tol': 1e-2,
    'conj_grad_iter': 100,
}

## Functions

In [38]:
func_x = lambda X: ce.cov_val(A, X)
func_beta = lambda X, S, beta, extra_param, extra_param_s: ce.cov_val(A, (1 - beta) * X + beta * S)
grad_x = lambda X, inv_X: ce.cov_grad(A, X, inv_X)
grad_beta = lambda X, S, beta, inv_X: ce.cov_grad(A, (1 - beta) * X + beta * S) # TODO: add inverse
hess_mult_x = lambda S, inv_X: ce.cov_hess_mult(S, inv_X)
hess_mult_vec_x = lambda S, trace_sum: ce.cov_hess_mult_vec(S, inv_X)
extra_func = lambda X: np.linalg.eigvalsh(X)
linear_oracle = lambda grad: ce.linear_oracle(grad)
prox_func = lambda X, L: ce.proj_map(X) #used for SCOPT

## Run FW

In [41]:
#run_alpha_policies = ["standard", "line_search", "icml", "backtracking"]
run_alpha_policies = ["backtracking","standard"]

for policy in run_alpha_policies:
    x, alpha_hist, Gap_hist, Q_hist, time_hist  = frank_wolfe(func_x,
                       func_beta,                                       
                       grad_x,
                       grad_beta,
                       hess_mult_x,
                       extra_func,                                                    
                       Mf,
                       nu,
                       linear_oracle,                                                    
                       X_0,
                       FW_params,
                       #hess=hess_x, 
                       #lloo_oracle=llo_oracle,                                                 
                       alpha_policy=policy,                                                    
                       eps=terminate_tol, 
                       print_every=1, 
                       debug_info=False)
    
    results[problem_name][policy] = {
        'x': x,
        'alpha_hist': alpha_hist,
        'Gap_hist': Gap_hist,
        'Q_hist': Q_hist,
        'time_hist': time_hist,
    }

********* Algorithm starts *********
f: 3743.139569 Gap: 5.684342e-14
iter = 1, stepsize = 2.277616851282888e-13, criterion = 2.275675155206688e-13, upper_bound=3743.1395692390934, lower_bound=3743.1395692390934, real_Gap=0.0
f: 3743.139569 Gap: 1.337526e-10
iter = 2, stepsize = 1.0718464902137264e-09, criterion = 2.275675155206688e-13, upper_bound=3743.1395692390934, lower_bound=3743.1395692390934, real_Gap=0.0
f: 3743.139569 Gap: 6.293077e-07
iter = 3, stepsize = 6.156070830360989e-10, criterion = 2.275675155206688e-13, upper_bound=3743.1395692390934, lower_bound=3743.1395692390934, real_Gap=0.0
f: 3743.139569 Gap: 9.906695e-07
iter = 4, stepsize = 9.691016148397771e-10, criterion = 2.275675155206688e-13, upper_bound=3743.1395692390934, lower_bound=3743.1395692390934, real_Gap=0.0
f: 3743.139569 Gap: 1.559532e-06
iter = 5, stepsize = 7.627895087400471e-10, criterion = 2.275675155206688e-13, upper_bound=3743.1395692390934, lower_bound=3743.1395692390934, real_Gap=0.0
f: 3743.139569 Ga

f: 3743.139569 Gap: 5.584035e-06
iter = 43, stepsize = 4.069853317881643e-17, criterion = 4.0549161250957866e-17, upper_bound=3743.139569239093, lower_bound=3743.1395692390934, real_Gap=-4.547473508864641e-13
f: 3743.139569 Gap: 5.584035e-06
iter = 44, stepsize = 4.069853317881643e-17, criterion = 4.0549161250957866e-17, upper_bound=3743.1395692390925, lower_bound=3743.1395692390934, real_Gap=-9.094947017729282e-13
f: 3743.139569 Gap: 5.584035e-06
iter = 45, stepsize = 4.069853400740862e-17, criterion = 4.0549161250957866e-17, upper_bound=3743.1395692390925, lower_bound=3743.1395692390934, real_Gap=-9.094947017729282e-13
f: 3743.139569 Gap: 5.584035e-06
iter = 46, stepsize = 4.069853400740862e-17, criterion = 4.0549161250957866e-17, upper_bound=3743.1395692390925, lower_bound=3743.1395692390934, real_Gap=-9.094947017729282e-13
f: 3743.139569 Gap: 5.584035e-06
iter = 47, stepsize = 4.069853400740862e-17, criterion = 4.0549161250957866e-17, upper_bound=3743.1395692390925, lower_bound=374

f: 3743.139569 Gap: 5.584035e-06
iter = 82, stepsize = 4.0698532350224255e-17, criterion = 4.0549161250957866e-17, upper_bound=3743.139569239092, lower_bound=3743.1395692390934, real_Gap=-1.3642420526593924e-12
f: 3743.139569 Gap: 5.584035e-06
iter = 83, stepsize = 4.0698532350224255e-17, criterion = 4.0549161250957866e-17, upper_bound=3743.139569239092, lower_bound=3743.1395692390934, real_Gap=-1.3642420526593924e-12
f: 3743.139569 Gap: 5.584035e-06
iter = 84, stepsize = 4.0698532350224255e-17, criterion = 4.0549161250957866e-17, upper_bound=3743.1395692390915, lower_bound=3743.1395692390934, real_Gap=-1.8189894035458565e-12
f: 3743.139569 Gap: 5.584035e-06
iter = 85, stepsize = 4.0698532350224255e-17, criterion = 4.0549161250957866e-17, upper_bound=3743.1395692390915, lower_bound=3743.1395692390934, real_Gap=-1.8189894035458565e-12
f: 3743.139569 Gap: 5.584035e-06
iter = 86, stepsize = 4.0698532350224255e-17, criterion = 4.0549161250957866e-17, upper_bound=3743.1395692390915, lower_b

iter = 24, stepsize = 0.07407407407407407, criterion = 0.07598498912739035, upper_bound=3743.1395692390934, lower_bound=3743.1395692390934, real_Gap=0.0
f: 6351.321185 Gap: 6.809200e+04
iter = 25, stepsize = 0.07142857142857142, criterion = 0.07320267068601957, upper_bound=3743.1395692390934, lower_bound=3743.1395692390934, real_Gap=0.0
f: 6386.250126 Gap: 7.337500e+04
iter = 26, stepsize = 0.06896551724137931, criterion = 0.07061697798394594, upper_bound=3743.1395692390934, lower_bound=3743.1395692390934, real_Gap=0.0
f: 6419.587739 Gap: 7.885367e+04
iter = 27, stepsize = 0.06666666666666667, criterion = 0.0682077741858456, upper_bound=3743.1395692390934, lower_bound=3743.1395692390934, real_Gap=0.0
f: 6451.442670 Gap: 8.452800e+04
iter = 28, stepsize = 0.06451612903225806, criterion = 0.06595758178318642, upper_bound=3743.1395692390934, lower_bound=3743.1395692390934, real_Gap=0.0
f: 6481.912726 Gap: 9.039800e+04
iter = 29, stepsize = 0.0625, criterion = 0.06385115759758413, upper_bo

f: 7098.877579 Gap: 4.856447e+05
iter = 69, stepsize = 0.027777777777777776, criterion = 0.028039223363857113, upper_bound=3743.1395692390934, lower_bound=3743.1395692390934, real_Gap=0.0
f: 7105.874719 Gap: 4.995370e+05
iter = 70, stepsize = 0.0273972602739726, criterion = 0.027651534599243154, upper_bound=3743.1395692390934, lower_bound=3743.1395692390934, real_Gap=0.0
f: 7112.628174 Gap: 5.136250e+05
iter = 71, stepsize = 0.02702702702702703, criterion = 0.027274421149855736, upper_bound=3743.1395692390934, lower_bound=3743.1395692390934, real_Gap=0.0
f: 7119.144430 Gap: 5.279087e+05
iter = 72, stepsize = 0.02666666666666667, criterion = 0.02690745610517382, upper_bound=3743.1395692390934, lower_bound=3743.1395692390934, real_Gap=0.0
f: 7125.429714 Gap: 5.423880e+05
iter = 73, stepsize = 0.02631578947368421, criterion = 0.02655023522921026, upper_bound=3743.1395692390934, lower_bound=3743.1395692390934, real_Gap=0.0
f: 7131.490006 Gap: 5.570630e+05
iter = 74, stepsize = 0.0259740259

In [27]:
grad = grad_x(X_0, np.linalg.inv(X_0))

In [28]:
grad

array([[-5.86000000e+02,  1.98246239e-01,  2.27265618e-01, ...,
         1.01218126e-01,  1.81443299e-01, -4.68225435e-03],
       [ 1.98246239e-01, -5.86000000e+02,  8.49994237e-01, ...,
        -1.87059762e-01,  1.82687279e-01, -5.51225886e-02],
       [ 2.27265618e-01,  8.49994237e-01, -5.86000000e+02, ...,
        -2.60526925e-02,  1.77413834e-01, -1.70621545e-01],
       ...,
       [ 1.01218126e-01, -1.87059762e-01, -2.60526925e-02, ...,
        -5.86000000e+02, -2.05254916e-01, -8.54251376e-02],
       [ 1.81443299e-01,  1.82687279e-01,  1.77413834e-01, ...,
        -2.05254916e-01, -5.86000000e+02, -5.12942325e-02],
       [-4.68225435e-03, -5.51225886e-02, -1.70621545e-01, ...,
        -8.54251376e-02, -5.12942325e-02, -5.86000000e+02]])

In [29]:
S = linear_oracle(grad)
S

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [30]:
x_nxt = X_0 + 2/3 * (S - X_0)

In [32]:
np.sum(abs(S))

1.0

In [33]:
x_nxt

array([[6.67234526e-01, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 5.67859171e-04, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 5.67859171e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        5.67859171e-04, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 5.67859171e-04, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 5.67859171e-04]])

In [39]:
x = X_0
f_vals = []
for i in range(100):
    f, inv_X = func_x(x)
    x = x - 0.01 * grad_x(x, inv_X)
    f_vals.append(f)

In [40]:
f_vals

[3743.1395692390934,
 2253.9285299320827,
 2101.179286305169,
 1948.5753972224338,
 1796.1892120512784,
 1644.1986204095851,
 1493.445091643542,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]